In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

# KNN(K nearest neighbor)

In [13]:
## 데이터 불러오기
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [14]:
## 데이터 탐색
print(df.info(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None 



In [15]:
rdf = df.drop(['deck','embark_town'], axis=1) # null이 많은 deck 컬럼 삭제, embarked와 중복되는 embark_town 컬럼 삭제
rdf.dropna(subset=['age'], how='any', axis=0, inplace=True) # age컬럼에 null이 존재하는 행 삭제
rdf

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,Third,woman,False,no,False
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,yes,True
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,yes,True


In [16]:
# embarked 컬럼은 null값이 2개이므로 null을 최빈값으로 대체
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax() # embarked컬럼에서 null을 제외하고 각 값의 개수를 세고, 최대개수의 인덱스를 반환
rdf['embarked'].fillna(most_freq, inplace=True)

# 데이터 기술통계 요약 확인
rdf.describe(include='all') # 범주형 데이터에 대한 통계가지 포함해서 전부 출력

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,714,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,556,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
## 피쳐 선택
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [18]:
# 범주형 데이터 숫자형으로 변환 - 원핫인코딩
onehot_sex = pd.get_dummies(ndf['sex']) # sex 컬럼의 값을 컬럼명으로 더미 변수 열 생성(male, female 2개의 열)
ndf = pd.concat([ndf, onehot_sex],axis=1) # 생성된 더미 변수 열을 기존 ndf와 열 기준으로 결합
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town') # embarked 컬럼의 값에 town을 붙여서 컬럼명으로 더미 변수 열 생성(town_C/Q/S 3개의 열)
ndf = pd.concat([ndf, onehot_embarked],axis=1)# 생성된 더미 변수 열을 ndf와 열 기준으로 결합
ndf.drop(['sex','embarked'],axis=1,inplace=True) # 기존 범주형데이터를 값으로 가진 컬럼 삭제
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [23]:
## 데이터 분할
X = ndf.drop(['survived'], axis=1) # 교재는 일일이 컬럼명으로 지정했으나 귀찮음
y = ndf['survived']

# 독립변수 정규화(normalization)
X = preprocessing.StandardScaler().fit(X).transform(X) # sklearn의 StandardScaler 사용

# train/test/ dataset separation
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

print(f'train data 개수 : {X_train.shape}')
print(f'test data 개수 : {X_test.shape}')

train data 개수 : (499, 9)
test data 개수 : (215, 9)


In [26]:
## 학습 및 분류 예측
knn = KNeighborsClassifier(n_neighbors=5) # k를 5로 설정하고 knn 분류 객체 생성
knn.fit(X_train,y_train) # train 데이터로 학습
y_hat = knn.predict(X_test) # test 데이터로 예측

print(y_hat[:10], '\n') # 예측값 중 10개 출력
print(y_test.values[:10]) # 실제값 중 10개 출력

[0 1 1 1 0 1 0 1 0 0] 

[0 1 1 1 0 1 1 1 0 0]


In [28]:
## 평가 지표 계산
# 혼동행렬
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
knn_matrix

array([[110,  16],
       [ 26,  63]], dtype=int64)

In [33]:
# 정밀도, 재현율, F1 score
acc = metrics.accuracy_score(y_test, y_hat)
print(f'accuracy = {acc}','\n')

prc = metrics.precision_score(y_test, y_hat)
print(f'precision = {prc}','\n')

rec = metrics.recall_score(y_test, y_hat)
print(f'recall = {rec}','\n')

f1 = metrics.f1_score(y_test, y_hat)
print(f'f1 score = {f1}','\n')

knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

accuracy = 0.8046511627906977 

precision = 0.7974683544303798 

recall = 0.7078651685393258 

f1 score = 0.7499999999999999 

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       126
           1       0.80      0.71      0.75        89

    accuracy                           0.80       215
   macro avg       0.80      0.79      0.79       215
weighted avg       0.80      0.80      0.80       215



# SVM(Support Vector Machine)

In [34]:
## 데이터를 불러와 학습 전까지 과정은 KNN과 동일

In [37]:
## 학습 및 예측
svm_model = svm.SVC(kernel='rbf') # svm 객체 생성, 커널(데이터를 벡터 공간으로 매핑하는 함수)=radial basis function 사용
svm_model.fit(X_train, y_train) # 학습
y_hat = svm_model.predict(X_test) # 예측

print(y_hat[:10], '\n') # 예측값 중 10개 출력
print(y_test.values[:10]) # 실제값 중 10개 출력

[0 1 1 1 0 1 0 1 0 1] 

[0 1 1 1 0 1 1 1 0 0]


In [38]:
## 평가 지표 계산
# 혼동행렬
svm_matrix = metrics.confusion_matrix(y_test, y_hat)
svm_matrix

array([[115,  11],
       [ 30,  59]], dtype=int64)

In [39]:
# 정밀도, 재현율, F1 score
acc = metrics.accuracy_score(y_test, y_hat)
print(f'accuracy = {acc}','\n')

prc = metrics.precision_score(y_test, y_hat)
print(f'precision = {prc}','\n')

rec = metrics.recall_score(y_test, y_hat)
print(f'recall = {rec}','\n')

f1 = metrics.f1_score(y_test, y_hat)
print(f'f1 score = {f1}','\n')

svm_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

accuracy = 0.8093023255813954 

precision = 0.8428571428571429 

recall = 0.6629213483146067 

f1 score = 0.7421383647798743 

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       126
           1       0.80      0.71      0.75        89

    accuracy                           0.80       215
   macro avg       0.80      0.79      0.79       215
weighted avg       0.80      0.80      0.80       215



# Decision Tree(의사결정나무)

In [56]:
## 데이터 준비
uci_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'
df = pd.read_csv(uci_path, header=None)
df

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [57]:
df.columns = ['id','clump','cell_size','cell_shape','adhesion','epithlial','bare_nuclei','chromatin','nermal_nucleoli','mitoses','class']
df.head() # 열 이름 지정 후 첫 5줄 확인

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nuclei,chromatin,nermal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [58]:
df.info() # 요약정보 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               699 non-null    int64 
 1   clump            699 non-null    int64 
 2   cell_size        699 non-null    int64 
 3   cell_shape       699 non-null    int64 
 4   adhesion         699 non-null    int64 
 5   epithlial        699 non-null    int64 
 6   bare_nuclei      699 non-null    object
 7   chromatin        699 non-null    int64 
 8   nermal_nucleoli  699 non-null    int64 
 9   mitoses          699 non-null    int64 
 10  class            699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [59]:
df.describe() # 기술통계 요약정보 확인

,id,clump,cell_size,cell_shape,adhesion,epithlial,chromatin,nermal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [60]:
# bare_nuclei 컬럼만 object이므로 확인 후 수정
print(df.bare_nuclei.unique()) # ?가 존재
print(df.bare_nuclei.value_counts()) # ?가 16개 존재 > 699개 중 16개이므로 삭제
df.bare_nuclei.replace('?',np.nan,inplace=True) # ?를 NaN으로 변경
df.dropna(subset=['bare_nuclei'], inplace=True) # NaN이 포함된 행 제거
df.bare_nuclei = df.bare_nuclei.astype('int64') # bare_nuclei 컬럼의 자료타입을 int64로 변경
df.describe()

['1' '10' '2' '4' '3' '9' '7' '?' '5' '8' '6']
1     402
10    132
2      30
5      30
3      28
8      21
4      19
?      16
9       9
7       8
6       4
Name: bare_nuclei, dtype: int64


,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nuclei,chromatin,nermal_nucleoli,mitoses,class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [65]:
## 속성 선택
X = df.drop(['id','class'], axis=1)
y = df['class']

# 데이터 정규화
X = preprocessing.StandardScaler().fit(X).transform(X)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

print(f'train data 개수 : {X_train.shape}')
print(f'test data 개수 : {X_test.shape}')

train data 개수 : (478, 9)
test data 개수 : (205, 9)


In [67]:
## 학습 및 검증
tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=5) # 의사결정나무 객체 생성, 분류기준은 엔트로피, 최대 깊이는 5
tree_model.fit(X_train, y_train) # 학습
y_hat = tree_model.predict(X_test) # 예측

print(y_hat[:10], '\n') # 예측값 중 10개 출력
print(y_test.values[:10]) # 실제값 중 10개 출력

[4 4 2 2 2 2 2 2 4 2] 

[4 4 2 2 2 2 2 2 4 2]


In [68]:
## 평가 지표
# 혼동행렬
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
tree_matrix

array([[125,   2],
       [  4,  74]], dtype=int64)

In [71]:
# 정밀도, 재현율, F1 score
acc = metrics.accuracy_score(y_test, y_hat)
print(f'accuracy = {acc}','\n')

prc = metrics.precision_score(y_test, y_hat, pos_label=2) # 타겟의 클래스가 0과1로 이루어지지 않은 경우, 뭐가 positive인지 몰라서 오류 발생
print(f'precision = {prc}','\n')

rec = metrics.recall_score(y_test, y_hat, pos_label=2) # 따라서 2나 4로 지정해줘야 수행이 정상적으로 됨
print(f'recall = {rec}','\n')

f1 = metrics.f1_score(y_test, y_hat, pos_label=2)
print(f'f1 score = {f1}','\n')

tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

accuracy = 0.9707317073170731 

precision = 0.9689922480620154 

recall = 0.984251968503937 

f1 score = 0.9765625 

              precision    recall  f1-score   support

           2       0.97      0.98      0.98       127
           4       0.97      0.95      0.96        78

    accuracy                           0.97       205
   macro avg       0.97      0.97      0.97       205
weighted avg       0.97      0.97      0.97       205

